# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   301.028180  0.753245  1234  301.781425  300.274935   603.562850   
6   821.381603 -0.502789  1234  820.878814  821.884393  1641.757628   
7   140.226102 -0.915085  1234  139.311016  141.141187   278.622033   
8   475.876066  0.484211  1234  476.360277  475.391855   952.720554   
9   110.628624  0.830928  1234  111.459552  109.797697   222.919104   
10  349.985350  0.503688  1234  350.489038  349.481663   700.978076   
11  448.985519  0.246561  1234  449.232080  448.738958   898.464160   
12  510.812545  0.472818  1234  511.285362  510.339727  1022.570725   
13  850.370098  0.393955  1234  850.764053  849.976142  1701.528106   
14   59.015548 -0.417579  1234   58.597969   59.433127   117.195937   

              f  
5    600.549869  
6   1643.768785  
7    282.282374  
8    950.783709  
9    219.595393  
10   698.963326  
11   897.477916  
12  1020.679453  
13  1699.952285  
14   118.866254

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -301.781425   300.274935
6   -820.878814   821.884393
7   -139.311016   141.141187
8   -476.360277   475.391855
9   -111.459552   109.797697
10  -350.489038   349.481663
11  -449.232080   448.738958
12  -511.285362   510.339727
13  -850.764053   849.976142
14   -58.597969    59.433127

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -301.781425   300.274935
6   -820.878814   821.884393
7   -139.311016   141.141187
8   -476.360277   475.391855
9   -111.459552   109.797697
10  -350.489038   349.481663
11  -449.232080   448.738958
12  -511.285362   510.339727
13  -850.764053   849.976142
14   -58.597969    59.433127

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-301.781425  ,  300.27493456],
       [-820.87881408,  821.88439269],
       [-139.31101629,  141.14118715],
       [-476.36027706,  475.3918546 ],
       [-111.45955217,  109.79769653],
       [-350.48903804,  349.48166288],
       [-449.23208021,  448.7389582 ],
       [-511.28536239,  510.3397267 ],
       [-850.76405325,  849.97614236],
       [ -58.59796851,   59.433127  ]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5     601.562850   -600.549869
6    1639.757628  -1643.768785
7     276.622033   -282.282374
8     950.720554   -950.783709
9     220.919104   -219.595393
10    698.978076   -698.963326
11    896.464160   -897.477916
12   1020.570725  -1020.679453
13   1699.528106  -1699.952285
14    115.195937   -118.866254

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf